In [3]:
import cred
import google.generativeai as genai

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Set up Gemini API key
## TODO: Fill in your Gemini API in the ""
genai.configure(api_key=cred.keys['GEMINI_API_KEY'])
model = genai.GenerativeModel(model_name="gemini-1.5-flash")  # No need to prefix with "models/"

# Check if you have set your Gemini API successfully
# You should see "Set Gemini API sucessfully!!" if nothing goes wrong.
try:
    model.generate_content(
      "test",
    )
    print("Set Gemini API sucessfully!!")
except:
    print("There seems to be something wrong with your Gemini API. Please follow our demonstration in the slide to get a correct one.")

Set Gemini API sucessfully!!


In [5]:
# 單次對話示範: 
def interact_summarization(prompt: str, article: str, temp = 1.0):
    '''
      * Arguments

        - prompt: the prompt that we use in this section

        - article: the article to be summarized

        - temp: the temperature parameter of this model. Temperature is used to control the output of the chatbot.
                The higher the temperature is, the more creative response you will get.
    '''
    input = f"{prompt}\n{article}"
    response = model.generate_content(
      input,
      generation_config=genai.types.GenerationConfig(temperature=temp),
      safety_settings=[
          {"category": "HARM_CATEGORY_HARASSMENT","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_HATE_SPEECH","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_DANGEROUS_CONTENT","threshold": "BLOCK_NONE",},
          ]
    )

    return [(input, response.text)]


prompt_textbox =  "Summary the article in 2 sentences"
article_textbox = "With house prices soaring, it's not easy finding somewhere to live. And this community has thrown in the towel. Meet Seattle's rolling neighborhood of RVs, where each unassuming vehicle is a capsule home. The unusual format has been captured in a series of photographs by visual journalist Anna Erickson. Meet Bud Dodson, 57, and welcome to his home: An RV in Seattle's SoDo where he watches over the parking lot in exchange for a spot . No place like home: John Warden, 52, has turned his $200 vehicle into his home after his apartment burned down years ago . There are around 30 drivers that float in and out of this parking lot in the SoDo (South of Downtown) area of the city in Washington State. One might not notice them in the mornings as hundreds of workers in the nearby factories, such as Starbucks, park up and rush into work. But on the weekends, as the rabble flocks back to their beds, this unique group remains. John Worden, 52, has been living in his vehicle for years since his apartment burned down and he was left homeless. He told Anna his car cost $200, and doesn't drive very well. But for a home, it's just about enough. Though plan on the outside, it is a Pandora's Box inside, Anna tells DailyMail.com. 'It was scattered with trinkets that he had been collecting over the years,' she explained, 'and a pile of beer cans that he was saving to turn in for money.' For work, he panhandles while helping people find parking spaces at Safeco Field stadium, where he used to be a cook. People come and go for work in the factories nearby, but on the weekend it is just the RV-dwellers that area left . Daily life: Here Bud can be seen preparing himself a barbecue on the gravel outside his capsule home, one of about 30 in the community . Eclectic: While Bud's RV is organized and functional, John's is full of trinkets and belongings dating back years . Alongside him - most of the time - is Bud Dodson, 57. While some are forced to move about regularly, Dodson, a maintenance man, looks after the parking lot in exchange for a semi-permanent spot. His home has its own unique stamp on it. 'He had really made the RV his home and taken good care of it,' Anna described. 'It was more functional [than John's] and a cleaner space with a bed, kitchen and bathroom.' Whether organized or eclectic, however, each one is home. 'None of them seem to want to move on,' Anna said. 'It's not perfect but they seem pretty content. Move in, move out: Some have agreements to stay, but others have to keep driving around to find a spot . John works as a panhandler at Safeco Fields stadium, where he used to work as a cook . He is content with his life in between the usual confines of society . Personal: To many this may just seem like a parking lot but for these men it is a very personal space . 'Bud is very grateful, he said the parking lot owner is just such a nice guy to let him live like this.' She came across them when she stopped to ask a seemingly homeless man for directions. 'We got talking,' she said, 'and he mentioned that he lived nearby in an RV. I went round to look and there was a whole bunch of them.' Curious, she spent about two months returning to the spot, meeting with the community and building their trust. 'These RVs are their homes so it's a very personal thing,' she explained."
temperature_slider = 0.7

summary = interact_summarization(prompt_textbox, article_textbox, temperature_slider)
print(summary)

[("Summary the article in 2 sentences\nWith house prices soaring, it's not easy finding somewhere to live. And this community has thrown in the towel. Meet Seattle's rolling neighborhood of RVs, where each unassuming vehicle is a capsule home. The unusual format has been captured in a series of photographs by visual journalist Anna Erickson. Meet Bud Dodson, 57, and welcome to his home: An RV in Seattle's SoDo where he watches over the parking lot in exchange for a spot . No place like home: John Warden, 52, has turned his $200 vehicle into his home after his apartment burned down years ago . There are around 30 drivers that float in and out of this parking lot in the SoDo (South of Downtown) area of the city in Washington State. One might not notice them in the mornings as hundreds of workers in the nearby factories, such as Starbucks, park up and rush into work. But on the weekends, as the rabble flocks back to their beds, this unique group remains. John Worden, 52, has been living i

In [6]:
#實際多次對話時，互動模式，需要用一個list，不停append，並且指定對話的腳色跟內容
def interact_customize(chatbot: list[tuple[str, str]], prompt: str ,user_input: str, temp = 1.0)->list:
    '''
    * Arguments

      - chatbot: the model itself, the conversation is stored in list of tuples

      - prompt: the prompt for your desginated task

      - user_input: the user input of each round of conversation

      - temp: the temperature parameter of this model. Temperature is used to control the output of the chatbot.
              The higher the temperature is, the more creative response you will get.

    '''
    try:
        messages = []

        for input_text, response_text in chatbot:
            messages.append({'role': 'user', 'parts': [input_text]})
            messages.append({'role': 'model', 'parts': [response_text]})

        messages.append({'role': 'user', 'parts': [prompt]})

        response = model.generate_content(
          messages,
          generation_config=genai.types.GenerationConfig(temperature=temp),
          safety_settings=[
          {"category": "HARM_CATEGORY_HARASSMENT","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_HATE_SPEECH","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT","threshold": "BLOCK_NONE",},
          {"category": "HARM_CATEGORY_DANGEROUS_CONTENT","threshold": "BLOCK_NONE",},
          ]
        )

        chatbot.append((user_input, response.text))

    except Exception as e:
        print(f"Error occurred: {e}")
        chatbot.append((user_input, f"Sorry, an error occurred: {e}"))
    return chatbot


In [10]:
chatbot = []
prompt = 'You are a poet'
user_input = "Hello, can you create a poet for me?"
chatbot = interact_customize(chatbot, prompt, user_input)

In [8]:
user_input = 'Sure, it is my honor to hear'
chatbot = interact_customize(chatbot, prompt, user_input)

In [11]:
chatbot

[('Hello, can you create a poet for me?',
  "The wind whispers secrets to the ancient trees,\nTheir leaves a rustling chorus on the breeze.\nThe sun dips low, a painter's fiery hand,\nAcross the canvas of the sleeping land.\n\nA lonely star, a diamond in the night,\nObserves the world in pale and silent light.\nThe moon ascends, a silver sickle bright,\nCasting long shadows in the fading light.\n\nAnd in the hush, a heart begins to beat,\nA rhythm soft, a cadence bittersweet.\nOf joys and sorrows, hopes and whispered fears,\nReflected in the passing of the years.\n\nFor life's a poem, etched in every line,\nA tapestry of moments, yours and mine.\nSo let us read it, with a grateful heart,\nAnd play our part, before we must depart.\n")]